In [ ]:
from tqdm.notebook import tqdm


os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [9]:
%run Model_setup.ipynb
%run Create_list_of_URLs.ipynb

['www.free.fr', 'www.google.fr', 'www.amazon.fr', 'www.lemonde.fr', 'www.lefigaro.fr', 'www.weborama.fr', 'www.orange.fr', 'www.leboncoin.fr', 'www.rfi.fr', 'www.francetvinfo.fr']


In [8]:
# Create list of URLs of specific dates
dictory_of_urls = go_over_websites(first_ten_websites)

www.free.fr https://web.archive.org/web/20200713180929/https://free.fr/ Success.
www.free.fr https://web.archive.org/web/20200813184803/http://free.fr/ Success.
www.free.fr https://web.archive.org/web/20200913181526/http://emmaus.saintmarcel@free.fr/ Success.
www.free.fr https://web.archive.org/web/20201013181344/https://free.fr/ Success.
www.free.fr https://web.archive.org/web/20201113183825/http://rmimausa@free.fr/ Success.
www.free.fr https://web.archive.org/web/20201213172351/https://free.fr/ Success.
www.google.fr https://web.archive.org/web/20200713182212/https://www.google.fr/ Success.
www.google.fr https://web.archive.org/web/20200813183344/https://www.google.fr/ Success.
www.google.fr https://web.archive.org/web/20200913184038/https://www.google.fr/ Success.
www.google.fr https://web.archive.org/web/20201013184605/https://google.fr/ Success.
www.google.fr https://web.archive.org/web/20201113181609/http://www.google.fr/ Success.
www.google.fr https://web.archive.org/web/2020121

In [12]:
print(dictory_of_urls)

{'www.free.fr': [('https://web.archive.org/web/20200713180929/https://free.fr/', (2020, 7)), ('https://web.archive.org/web/20200813184803/http://free.fr/', (2020, 8)), ('https://web.archive.org/web/20200913181526/http://emmaus.saintmarcel@free.fr/', (2020, 9)), ('https://web.archive.org/web/20201013181344/https://free.fr/', (2020, 10)), ('https://web.archive.org/web/20201113183825/http://rmimausa@free.fr/', (2020, 11)), ('https://web.archive.org/web/20201213172351/https://free.fr/', (2020, 12))], 'www.google.fr': [('https://web.archive.org/web/20200713182212/https://www.google.fr/', (2020, 7)), ('https://web.archive.org/web/20200813183344/https://www.google.fr/', (2020, 8)), ('https://web.archive.org/web/20200913184038/https://www.google.fr/', (2020, 9)), ('https://web.archive.org/web/20201013184605/https://google.fr/', (2020, 10)), ('https://web.archive.org/web/20201113181609/http://www.google.fr/', (2020, 11)), ('https://web.archive.org/web/20201213162504/https://www.google.fr/', (20

In [11]:
createJSON("urls.json", dictory_of_urls)

In [24]:
def crawling(url):
    """
    Main function to extract iframes and active elements from a web page and pass them to an ML model.

    Args:
        url (str): The URL of the web page.
    """
    driver = setup_firefox_driver()
    driver.get(url)
    sleep(5)
    html = driver.page_source

    list_cookies = []
    
    soup = BeautifulSoup(html, 'html.parser')
    
    iframes = soup.find_all(['div', 'iframe'])
    active_elements = soup.find_all(['a', 'button', 'input', 'yt-formatted-string', 'p', 'span', 'form'])
    
    #button_elements = soup.find_all(['button', 'input', 'span', 'a'])
    button_elements = soup.find_all(['button', 'input[type="submit"]', 'a'])
    

    # Extract text content from iframes and active elements
    text_content = extract_text_from_elements(active_elements)
        
    button_text = extract_text_from_elements(button_elements)
    #text_content.extend(extract_text_from_iframes(iframes, driver))
    
    #check = search_dialog(iframes, text_content)
    search_frames(iframes, text_content, button_text)
    
    list_cookies = driver.get_cookies()
    driver.quit()
    dialog_result = pass_to_ml_model_dialog(text_content)
    if dialog_result != "not found":
        return (dialog_result, pass_to_ml_model_buttons(button_text)), stored_cookies
    return "not found", stored_cookies

In [25]:
webs = []
for web in dictory_of_urls:
    webs.append(web)

In [27]:
stored_cookies = dict()
for web in tqdm(webs):
    stored_cookies[web] = dict()
    for url_to_visit in dictory_of_urls[web]:
        results_web, cookies = crawling(url_to_visit[0])
        stored_cookies[web][url_to_visit] = cookies
        if not (results_web == "not found"):
            print(url_to_visit)
            print(results_web)
            break
        else:
            print("{} {}".format(url_to_visit, results_web))
print(stored_cookies)

  0%|          | 0/10 [00:00<?, ?it/s]

('https://web.archive.org/web/20200713180929/https://free.fr/', (2020, 7)) not found
('https://web.archive.org/web/20200813184803/http://free.fr/', (2020, 8)) not found


NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:187:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:613:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:821:39
despatch@chrome://remote/content/marionette/server.sys.mjs:320:40
execute@chrome://remote/content/marionette/server.sys.mjs:291:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:264:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:265:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:496:20
